In [0]:
##### loading data #####
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# train_dir = "/gdrive/My Drive/kaggle_dataset/train/"
# val_dir = "/gdrive/My Drive/kaggle_dataset/val/"
# test_dir = "/gdrive/My Drive/kaggle_dataset/test/"

train_dir = "/gdrive/My Drive/trainProbMaps/"
val_dir = "/gdrive/My Drive/valProbMaps/"
test_dir = "/gdrive/My Drive/testProbMaps/"


img_width = 600
img_height = 450

# img_width = 512
# img_height = 496

batch_size = 5
channels = 3
epochs = 50
nb_train_samples = 408
nb_valid_samples = 149
nb_test_samples = 192
num_classes = 2

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)             
valid_datagen = ImageDataGenerator(rescale=1./255)    
test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)   

valid_generator = valid_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True) #weight toward one class or another

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

Using TensorFlow backend.


Found 408 images belonging to 2 classes.
Found 149 images belonging to 2 classes.
Found 192 images belonging to 2 classes.


In [0]:
import numpy as np 
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization 
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
from keras import layers
from keras.callbacks import TensorBoard, Callback
from keras.applications import VGG16, resnet50
from keras import optimizers
import keras


class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)


pretrained_VGG16 = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_height, img_width, channels))

# pretrained_resnet50 = resnet50(weights='imagenet',
#                       include_top=False,
#                       input_shape=(img_height, img_width, channels))

# def downStreamModel(train_generator, valid_generator, test_generator, pt_model, optimizer):
#     base_model = pt_model
#     x = base_model.output
#     convDS = Conv2D(256, kernel_size=(3, 3), padding='valid')(x)
#     flatDS = Flatten()(x)
#     dropOutDS = Dropout(0.75)(x)
#     predictions = Dense(num_classes, activation='softmax')(x)
    
#     model = Model(inputs=base_model.input, outputs=predictions)
    
#     # Train top layers only
#     for layer in base_model.layers:
#         layer.trainable = False
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=optimizer,
#                   metrics=['accuracy'])
#     callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1)]
#     model.summary()
    
#     history = model.fit_generator(
#                 train_generator,
#                 steps_per_epoch=nb_train_samples / batch_size,
#                 epochs=epochs,
#                 validation_data=valid_generator,
#                 validation_steps=nb_valid_samples / batch_size,
#                 shuffle=True,
#                 verbose=1,
#                 callbacks=callbacks_list+[MetricsCheckpoint('logs')])
    
#     score = model.evaluate_generator(test_generator, verbose=0, steps=nb_test_samples / batch_size,)
#     print('accuracy:', score[1])
#     # for generating confusion matrix and more statistics
#     # y_pred = model.predict_generator(test_generator)
#     # print(sklearn.metrics.classification_report(np.where()))
#     return model

def downStreamModel(train_generator, valid_generator, test_generator, pt_model, optimizer):
    base_model = pt_model
    x = base_model.output
    convDS = Conv2D(256, kernel_size=(3, 3), padding='valid')(x)
    flatDS = Flatten()(convDS)
    dropOutDS = Dropout(0.75)(flatDS)
    predictions = Dense(1, activation='softmax')(dropOutDS)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Train top layers only
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1)]
    model.summary()
    
    history = model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples / batch_size,
                epochs=epochs,
                validation_data=valid_generator,
                validation_steps=nb_valid_samples / batch_size,
                shuffle=True,
                verbose=1,
                callbacks=callbacks_list+[MetricsCheckpoint('logs')])
    
    convDenseModel =  Model(inputs=base_model.input, outputs=flatDS)
    
    predictionsConvDense = convDenseModel.predict_generator(test_generator, verbose=0, steps=nb_test_samples / batch_size)
    
    
    score = model.evaluate_generator(test_generator, verbose=0, steps=nb_test_samples / batch_size,)
    print('accuracy:', score[1])
    # for generating confusionmatrix and more statistics
    # y_pred = model.predict_generator(test_generator)
    # print(sklearn.metrics.classification_report(np.where()))
    return model, predictionsConvDense


In [0]:
optimizer1 = keras.optimizers.Adam()
optimizer2 = keras.optimizers.RMSprop(lr=0.0001/2.0)

#convDenseModel =  Model(inputs=base_model.input, outputs=flatDS)

#model = downStreamModel(train_generator, valid_generator, test_generator, pretrained_VGG16, optimizer1)
model, predictionsConvDense = downStreamModel(train_generator, valid_generator, test_generator, pretrained_VGG16, optimizer1)

print(predictionsConvDense.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 450, 600, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 450, 600, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 450, 600, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 225, 300, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 225, 300, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 225, 300, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 150, 128)     0         
__________

In [0]:
print(predictionsConvDense.shape)

#features = predictionsConvDense.predict()

from google.colab import files
# test_features = np.reshape(test_features, (192, 7*7*512))
# print(test_features.shape)
#np.savetxt('LabelsTest2.txt', test_labels)
#files.download('LabelsTest2.txt')
np.savetxt('VGG_RF_FeaturesTest2.txt', predictionsConvDense)
files.download('VGG_RF_FeaturesTest2.txt')

(192, 49152)


----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 42490, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
def extract_features(dataset_type, sample_count):
    features = np.zeros(shape=(sample_count, 12, 16, 64))
    labels = np.zeros(shape=(sample_count))
    i = 0
    if dataset_type == "train":
        for inputs_batch, labels_batch in train_generator:
            features_batch = convDenseModel.predict(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            if i * batch_size >= sample_count:
                break   
    elif dataset_type == "valid":
        for inputs_batch, labels_batch in valid_generator:
            features_batch = convDenseModel.predict(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            if i * batch_size >= sample_count:
                break
    else:
        for inputs_batch, labels_batch in test_generator:
            features_batch = convDenseModel.predict(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            if i * batch_size >= sample_count:
                break
    return features, labels

train_features, train_labels = extract_features("train", nb_train_samples)
valid_features, valid_labels = extract_features("valid", nb_valid_samples)
test_features, test_labels = extract_features("test", nb_test_samples)


print(train_features.shape, train_labels.shape)
print(valid_features.shape, valid_labels.shape)
print(test_features.shape, test_labels.shape)

NameError: ignored

In [0]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples / batch_size,
                epochs=1,
                validation_data=valid_generator,
                validation_steps=nb_valid_samples / batch_size,
                shuffle=True,
                verbose=1,
                callbacks=callbacks_list+[MetricsCheckpoint('logs')])

score = model.evaluate_generator(test_generator, verbose=0, steps=1)
print('accuracy:', score[1])

NameError: ignored